In [1]:
from pyspark import SparkConf,SparkContext
conf = SparkConf()
conf.set("spark.master","local[2]")
conf.set("spark.app.name","sparkapp")
sc = SparkContext(conf=conf)
sc

<SparkContext master=local[2] appName=sparkapp>

In [2]:
sc.uiWebUrl

u'http://ip-172-31-26-204.us-east-2.compute.internal:4040'

In [18]:
from pyspark.sql import SQLContext

sqlc = SQLContext(sc)  # To work with dataframe API this becomes essential.

rd1 = sc.textFile("/home/ubuntu/sample.txt")  #By default, every ecord will be a string
print(rd1.collect())  #r1 is collection of strings
rd2 = rd1.map(lambda x:x.split(",")) #split can be done with a single character or a string
print(rd2.collect())   #each records is list of strings

rd3 = rd2.map(lambda x:(int(x[0]),x[1][1:],float(x[2])))
print(rd3.collect()) #re is tuple of (int,string)

#we can create a dataframe from the above RDD

df1 = rd3.toDF(["id","city","score"])
print(df1.show())   
print(df1.printSchema())

[u'123,#hyderabad,234.45', u'234,#chennai,345.45', u'345,#delhi,222.0', u'456,#bangalore,343.5']
[[u'123', u'#hyderabad', u'234.45'], [u'234', u'#chennai', u'345.45'], [u'345', u'#delhi', u'222.0'], [u'456', u'#bangalore', u'343.5']]
[(123, u'hyderabad', 234.45), (234, u'chennai', 345.45), (345, u'delhi', 222.0), (456, u'bangalore', 343.5)]
+---+---------+------+
| id|     city| score|
+---+---------+------+
|123|hyderabad|234.45|
|234|  chennai|345.45|
|345|    delhi| 222.0|
|456|bangalore| 343.5|
+---+---------+------+

None
root
 |-- id: long (nullable = true)
 |-- city: string (nullable = true)
 |-- score: double (nullable = true)

None


In [41]:
from pyspark.sql.types import StructType,IntegerType,StringType,StructField

cancer_schema = StructType([StructField("Id",IntegerType()),StructField("NameOfPatient",StringType()),StructField("Age",IntegerType()),StructField("Gender",StringType()),
                           StructField("State",StringType()),StructField("Symptoms",StringType()),StructField("Diag",StringType()),
                           StructField("Can",StringType()),StructField("Cancersc",StringType()),StructField("Stage",IntegerType()),
                           StructField("Treatment",StringType()),StructField("Survival",StringType())])
cancer_schema

df2 = sqlc.read.format("csv").option("header","True").schema(cancer_schema).load("/home/ubuntu/Datasets/cancerdata.csv") #this works when the deleimter has exactly one character
# print(df2.show(5))
# print(df2.printSchema())
 
# df2.select("ID","State").show(10)  #DSL Domaina specific Language
# df2.where('Age>50').show(10)

#To use pure SQL sytatemts on DF, create a table out of dataframe 
# df2.registerTempTable("cancertable")
# sqlc.sql("select * from cancertable where Age=50").show(5)



In [46]:
df_json = sqlc.read.format("json").load("/home/ubuntu/Datasets/companies.json")
# print(df_json.printSchema())
# print(df_json.show(5))
print(len(df_json.columns))

42
